# Cable Driven Parallel Robot 2T Model Tests
#### Created by: Alexander Herdt

This notebook is designed to train RL-agents for control of a cable driven parallel robot (CDPR) after cable break.  The CDPR is described as an RL-environment and has two translational degrees of freedom (2T) and a point mass endeffector/platform in a YZ-coordinate system. The specifications are derived by simplification of the CABLAR robot developed by the Chair of Mechatronics of the University of Duisburg-Essen. 

The following implementation is primarily based on the stable-baselines3 package which uses OpenAI gym as a basis for creating RL environments and Pytorch for training models with GPU integration. Furthermore Tensorboard can be used as a tool for observing leaning processes. Please refer to https://stable-baselines3.readthedocs.io/en/master/index.html for additional information.

## Imports

In [249]:
import json
import datetime
import pickle
import timeit
import numpy as np
from cdpr.utils import *
from environment import states
from environment.environment import EmergencyStrategyCDPR
from environment.observations import *
from environment.actions import Action, cable_forces_action, cable_force_gradients_action, platform_wrench_action
from environment.rewards import *
from environment.tasks import stop_pointmass_task
from evaluation import evaluation
from evaluation.visualize import *
from simulation.cablar import cablar1r2t, cablar2t
from simulation.agents import RLAgent
from typing import Dict, List, Union
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3 import SAC, PPO
from sb3_contrib import TQC
from stable_baselines3.common.env_checker import check_env

### CDPR Setup
Specify a cable driven parallel robot that is an instance of the CDPR class and an array coordinates.  
The coordinate array is necessary for workspace calculation and initialization of environment.

In [16]:
cdpr = cablar2t # CDPR Modell
env_kwargs = {"goal":np.array([[1.47],[1.38]])}
cables_idx = [0,2,3] # which cables should be used i.e. a cable that is not specified is considered "broken"
Ts = 0.005 # timestep 
max_episode_timesteps = 10000
# Coordinate grid for workspace calculation and initialization of environment
Y = np.linspace(-5,5,100) 
Z = np.linspace(0,5,100)
# PHI = np.deg2rad(np.arange(-30,30,1))
# PHI = np.array([0])
coordinates = create_grid_coordinates([Y,Z])

### Observations
Define all observation sets and their limits with the `Observation` class.  
All observations will be normalized by default to the interval __[-1,1]__ if you do not specify otherwise.  
Please use the `ObservationFunction` class and specify which variables will be observed if you would like to define a new observation function.  
Any observable variables should be defined as `StateVariable` or `DerivedVariable` object.

In [4]:
# Observe [pose, spatial_velocities, cable_forces] 
obs0 = Observation(obsfunc0,
                   low=[-5,0,-100,-100,150,150,150],
                   high=[5,5,100,100,2500,2500,2500])
# Observe [pose, velocities, cable_forces, position_error] 
obs1 = Observation(obsfunc1,
                   low=[-5,0,-100,-100,150,150,150,0,0],
                   high=[5,5,100,100,2500,2500,2500,10,5])

# Observe [pose, velocities, spatial_accelerations, cable_forces] 
obs2 = Observation(obsfunc2,
                   low=[-5,0,-100,-100,-100,-100,150,150,150],
                   high=[5,5,100,100,100,100,2500,2500,2500])

# Observe [pose, velocities, cable_forces, cable_lengths]
obs3 = Observation(obsfunc3,
                   low=[-5,0,-100,-100,150,150,150,0,0,0],
                   high=[5,5,100,100,2500,2500,2500,10,10,10])

# Observe [pose, spatial_velocities, cable_forces, steps] 
obs4 = Observation(obsfunc1,
                   low=[-5,0,-100,-100,150,150,150,0],
                   high=[5,5,100,100,2500,2500,2500,10000])

### Actions
Define actions as action functions and action spaces that will be used by the Agent.  
If new actions need to be defined please use the `Action` class.

In [5]:
act0 = cable_forces_action
act1 = cable_force_gradients_action
act2 = platform_wrench_action

### Rewards
Define rewards and the parameter values that should be tested.  
The parameter values should be in a dict of the form `{"parameter_name_0":value, "parameter_name_1":value, ...}`.  
If new rewards need to be defined please use the `Reward` class.

In [6]:
rew0 = neg_v_reward
rew_params0 = {"c1":[1,0.1,0.01],"c2":[1,2]}
rew1 = pos_v_reward
rew_params1 = {"c1":[1,0.1,0.01],"c2":[1,2]}
rew2 = neg_exp_v_reward
rew_params2 = {"c1":[1,0.1,0.01]}
rew3 = pos_exp_v_reward
rew_params3 = {"c1":[1,0.1,0.01]}

### Tasks
Define a task as a set of functions that determine whether termination criteria are met during an eposide.
If new tasks need to be defined please use the `Task` class.

In [7]:
task0 = stop_pointmass_task

### Definition of tests
Specify each test as a dictiionary `{"obs":your_observation, "act": your_action, "rew": your_reward, "rew_params": paramters_to_test, "task":your_task}`.  

In [242]:
tests = {
        0:{"obs":obs0,
           "act":act1,
           "rew":rew0,
           "rew_params":rew_params0,
           "task":task0},
        1:{"obs":obs0,
           "act":act1,
           "rew":rew1,
           "rew_params":rew_params1,
           "task":task0},
        2:{"obs":obs0,
           "act":act1,
           "rew":rew2,
           "rew_params":rew_params2,
           "task":task0},
        3:{"obs":obs0,
           "act":act1,
           "rew":rew3,
           "rew_params":rew_params3,
           "task":task0},
        }

## Testing
The testing procedure creates a folder in `models_path` that is named after the current datetime and is referred to as `model_path`. This folder contains a `setup_info.json` file and separate folders for every test.

Additional info:
- `setup_info.json` describes the configuration of the CDPR simulation and the foldernames for the different tests (a number `t_j` consisting of a testnumber `t` and parameter combination `j`).  
- Each test will be repeated once for every seed specified in `seeds` to account for the effect of random initialisation
- test folders will contain a trained model that was optained after  `total_timesteps` and a model that achieved the result during evaluations
- Evaluations are run after every `eval_freq` timesteps for `n_eval_episodes`
- Training logs will be stored in `model_path`+`logs`
- Training logs can accessed with **Tensorboard** by opening a python command prompt in the `model_path` folder and entering `tensorboard --logdir logs`

### Specify tests and configurations
The tests use default Hyperparameters if not specified otherwise.  
TQC: https://sb3-contrib.readthedocs.io/en/master/modules/tqc.html

In [ ]:
algo = TQC
total_timesteps = 1e6 # number of training timesteps
eval_freq = 100000 # number of timesteps before each evaluation
n_eval_episodes = 30 # number of episodes to run during evalutation
use_gSDE = False
n_envs = 1
config = {}

In [ ]:
seeds = [1,42,777]
testing_coordinates = coordinates
tests_now = list(tests.keys())
tests_path = os.path.join(os.getcwd(),"tests")
models_folder = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
# tests_now = [0,1,2]
# tests_path = ""
# models_folder = ""
model_path = os.path.join(tests_path, models_folder)
os.makedirs(model_path, exist_ok = True)

### Run testing procedures

In [ ]:
log_path = os.path.join(model_path,"logs")
setup_info = os.path.join(model_path, 'setup_info.txt')
setup_info = {"name":models_folder,
              "cables": cables_idx,
              "DoF":"2T",
              "Ts":Ts,
              "total_timesteps":total_timesteps}

test_params = {}
test_infos = {}

model_num = 1
for t, test_num in enumerate(tests_now):
    test = tests_now[test_num]
    obs = test["obs"]
    act = test["act"]
    rew = test["rew"]
    task = test["task"]
    test_infos[t] = {"obs_desc":obs.description,
                     "act_desc":act.description,
                    "rew_desc": rew.description,
                    "num":{}}
    rew_params = list(test["rew_params"].values())
    
    for i in range(len(rew_params)):
        param = rew_params[i]
        for j in range(len(param)):
            param_value = param[j]
            if type(param_value) == np.ndarray:
                rew_params[i][j] = param_value.tolist()
    if rew_params:
        param_combos = list(itertools.product(*rew_params))
    else:
        param_combos = [1]
    j = 0    
    for params in param_combos:
        test_infos[t]["num"][f"{t}_{j}"] = dict(zip(test["rew_params"].keys(),params))
        test_infos[t]["num"][f"{t}_{j}"]["tb_log_num"] = {}
        for seed in seeds:
            test_infos[t]["num"][f"{t}_{j}"]["tb_log_num"][seed] = model_num
            model_num +=1
        j += 1
    
setup_info["test_infos"] = test_infos
with open(os.path.join(model_path, "setup_info.json"),"w") as json_file:
    json.dump(setup_info, json_file, indent=4, separators = (",",":"))

all_results = {}

## Test Loop

In [ ]:
all_results = {}
mpl.use("Agg")
for t in tests_now:
    test = tests[t]
    obs= test["obs"]
    act = test["act"]
    rew = test["rew"]
    if rew_params:
        param_combos = list(itertools.product(*list(test["rew_params"].values())))
    else:
        param_combos = [1]
        
    for j, params in enumerate(param_combos):
        rew_params = dict(zip(test["rew_params"].keys(),params))
        env = EmergencyStrategyCDPR(
            cdpr=cdpr,
            cables_idx=cables_idx,
            Ts=Ts,
            max_episode_timesteps=max_episode_timesteps,
            coordinates=coordinates,
            observation=obs,
            action=act,
            reward=rew,
            task=task,
            rew_params=rew_params,
            **env_kwargs
            )
        
        for seed in seeds:
            # Vectorize environment for faster execution and add a Monitor wrapper for evaluation 
            vec_env = make_vec_env(lambda: env,
                                    seed = seed,
                                    monitor_kwargs=dict(info_keywords=("is_success",)),
                                    n_envs=n_envs)
            # Save path for best evaluated models during training
            savedir = os.path.join(model_path, f"{t}_{j}",str(seed))
            best_model_save_path = os.path.join(savedir, f"best")
            model_save = os.path.join(savedir,"Cdpr.zip")
            eval_env = make_vec_env(lambda: env)
            eval_callback = EvalCallback(eval_env, n_eval_episodes=n_eval_episodes,
                                        best_model_save_path=best_model_save_path,
                                        log_path=best_model_save_path, eval_freq=eval_freq,
                                        deterministic=True, render=False)
            vec_env.seed(seed)
            model = algo('MlpPolicy', 
                            vec_env, 
                            device='cuda',
                            verbose=0, 
                            seed = seed,
                            tensorboard_log=log_path,
                            **config)
            
            # Train the agent
            start = datetime.datetime.now()
            print(f"Begin training Model {t}_{j} seed {seed}. Start: {start.strftime('%d.%m.%Y %H:%M:%S')}")
            begin = timeit.default_timer()
            model.learn(total_timesteps=int(total_timesteps), callback=eval_callback)
            finish = timeit.default_timer()
            end = datetime.datetime.now()
            duration = finish-begin
            print(f"Done training Model {t}_{j} seed {seed}. End: {end.strftime('%d.%m.%Y %H:%M:%S')}")
            print(f"Duration: {end-start}")
            model.save(model_save)
            
            test_data = test_coordinates(env, model, coordinates)
            with open(os.path.join(savedir,'test_data.pkl'), 'wb') as file:
                pickle.dump(test_data, file)
                    
            successes = np.array(test_data["successes"])
            failures = np.array(test_data["failures"])
            timeouts = np.array(test_data["timeouts"])
            # fig = visualize_tests_2D(env, successes, failures, timeouts)
            # plt.savefig(os.path.join(savedir,"CDPR2T_.png"), format="png",dpi=150)
            # plt.close()
            test_table = {
                "Y":[], 
                "Z":[],
                "success":[],
                "timeout":[],
                "failure":[],
                "duration":[],
                "brake_time":[],
                "deceleration_time":[],
                'rms_jerk':[],
                # 'max_velocity':[],
                # 'rms_velocity':[],
                'trajectory_length':[],
                'trajectory_distance':[],
                'relative_length':[],
                'rms_forces':[],
                'rms_force_gradients':[],
                # 'max_force':[],
                'max_force_delta':[],
                'continuity_cost':[], 
                'df_continuity_cost':[]}
            # if "CDPR2T_new.xlsx" in files:
            #     test_table = pd.read_excel(os.path.join(savedir,"CDPR2T_new.xlsx"))
            # else:
            for k, trajectory in enumerate(test_data["trajectories"]):
                try:
                    poses, velocities, accelerations, forces, success, duration, done = trajectory.values()
                except:
                    poses, velocities, accelerations, forces, success, duration, done, metrics = trajectory.values()
                    
                translation_metrics = evaluation.evaluate_translation_trajectory(poses, velocities, accelerations)
                force_metrics = evaluation.evaluate_force_trajectory(forces)
                cc_mean = evaluation.continuity_cost(forces, env.f_min, env.f_max)
                delta_f = np.diff(forces, axis=0)
                brake_time = evaluation.find_final_threshold_index(np.linalg.norm(velocities,axis=1),0.1)*env.Ts
                deceleration_time = evaluation.find_not_rising_index(np.linalg.norm(velocities,axis=1))*env.Ts
                if len(delta_f)>1:
                    df_cc_mean = evaluation.continuity_cost(delta_f, -100, 100)
                else:
                    df_cc_mean = 0
                metrics = {}
                metrics.update(translation_metrics)
                metrics.update(force_metrics)
                metrics.update({"continuity_cost":cc_mean, 
                                "df_continuity_cost":df_cc_mean,})
                trajectory.update({"metrics":metrics})
                # Viz
                y = np.round(poses[0,0])
                z = np.round(poses[0,1])
                test_table["Y"].append(y)
                test_table["Z"].append(z)
                test_table["success"].append(success)
                test_table["timeout"].append(not done)
                test_table["failure"].append(not success and done)
                test_table["duration"].append(duration)
                test_table["brake_time"].append(brake_time)
                test_table["deceleration_time"].append(deceleration_time)
                for metric, value in metrics.items():
                    if len(metric)==2:
                        test_table[metric[0]].append(value[0])
                        test_table[metric[1]].append(value[1])
                    else:
                        test_table[metric].append(value)
                if k%2 == 0:
                    fig1 = visualize_forces(env, poses[0], Ts, forces)
                    fig2 = visualize_velocities(env, poses[0], Ts, velocities)
                    fig3 = visualize_poses(env, Ts, poses)
                    figs = [fig1, fig2, fig3]
                    fig_combi = combine_figures(figs)
                    plt.savefig(os.path.join(savedir,(f"forces_velocities_positions_{y}_{z}.png")), format="png",dpi=150)
                    plt.close('all')

            test_table = pd.DataFrame(test_table)
            with pd.ExcelWriter(os.path.join(savedir, "CDPR2T_new.xlsx")) as writer:
                df = pd.DataFrame(test_table)
                df.to_excel(writer, sheet_name= "test",engine="xlsxwriter")
            result = {
                "success_rate": sum(test_table["success"])/len(test_table["success"]),
                "failure_rate": sum(test_table["failure"])/len(test_table["failure"]),
                "timeout_rate": sum(test_table["timeout"])/len(test_table["timeout"]),
                "mean_jerk": np.mean(test_table["rms_jerk"]),
                # "mean_velocity": np.mean(test_table["rms_velocity"]),
                # "max_velocity": np.max(test_table["max_velocity"]),
                "mean_duration": np.mean(test_table["duration"]),
                "mean_success_duration": np.mean(test_table["duration"][test_table["success"]]),
                "mean_brake_time":np.mean(test_table["brake_time"][test_table["success"]]),
                "mean_deceleration_time_time":np.mean(test_table["deceleration_time"][test_table["success"]]),
                "mean_relative_length": np.mean(test_table["relative_length"][test_table["success"]]),
                "mean_forces": np.mean(test_table["rms_forces"]),
                "mean_max_force_delta": np.mean(test_table["max_force_delta"]),
                "mean_force_delta": np.mean(test_table["rms_force_gradients"]),
                "mean_continuity_cost": np.mean(test_table["continuity_cost"]),
                "mean_df_continuity_cost": np.mean(test_table["df_continuity_cost"]),
                "average_distance_to_centroid": evaluation.average_distance_to_centroid(successes)
                }